#### **Setup Python**

In [1]:
# Importação - Bibliotecas Python
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import glob
import warnings
import unicodedata
import re
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Remoção de avisos do Python
warnings.filterwarnings("ignore")

#### **Variáveis - Data Preprocessing TOTVS**

In [21]:
# Variável que armazena o caminho (path) das bases de dados presentes na camada raw
path_raw = "../data/raw"

# Variável que armazena o caminho (path) da camada "processed" onde as bases de dados processadas serão armazenadas inicialmente
path_processed = "../data/processed"

#### **Funções - Data Preprocessing TOTVS**

In [3]:
# Função para padronizar strings: maiúsculas, remover acentos e espaços extras
def padronizar_texto(coluna):
    return (
        coluna.astype(str)
              .str.strip()
              .str.upper()
              .apply(lambda x: unicodedata.normalize("NFKD", x)
                      .encode("ascii", errors="ignore").decode("utf-8"))
    )

# Função para calcular a idade em meses
def idade_contrato_meses(data_assinatura):
    if pd.isnull(data_assinatura):
        return np.nan
    delta = relativedelta(pd.Timestamp.today(), data_assinatura)
    return delta.years * 12 + delta.months

# Função que classifica a nota nps
def classificar_nps(nota):
    if nota >= 9:
        return "PROMOTOR"
    elif nota >= 7:
        return "NEUTRO"
    else:
        return "DETRATOR"

def inserir_dataframe_postgres(df, nome_tabela, if_exists='replace', chunksize=100000):
    # Carrega variáveis de ambiente
    load_dotenv(dotenv_path="../.env")

    # Dados de conexão
    usuario = os.getenv('DB_USER')
    senha = os.getenv('DB_PASSWORD')
    host = os.getenv('DB_HOST')
    porta = os.getenv('DB_PORT')
    banco = os.getenv('DB_NAME')

    # Verifica se as variáveis estão preenchidas
    if None in [usuario, senha, host, porta, banco]:
        raise ValueError("Alguma variável de ambiente do banco não está definida.")

    # Cria engine de conexão
    conn_string = f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}'
    engine = create_engine(conn_string)

    # Envia o DataFrame para o banco
    try:
        df.to_sql(
            name=nome_tabela,
            con=engine,
            if_exists=if_exists,
            index=False,
            chunksize=chunksize,
            method='multi'  # mais eficiente para lotes
        )
        print(f"Tabela '{nome_tabela}' inserida com sucesso!")
        print(f"Total de linhas: {df.shape[0]:,} | Total de colunas: {df.shape[1]}\n")
    except Exception as e:
        print(f"❌ Erro ao inserir '{nome_tabela}': {e}")

#### **Extract - Extração Dados Brutos (Raw)**

In [ ]:
# Leitura das bases de dados pertencentes há camada "Row"

# Base de dados "dados_clientes.csv"
dados_clientes = pd.read_csv(f"{path_raw}/dados_clientes.csv", sep=";")

# Base de dados "contratacoes_ultimos_12_meses.csv"
contratacoes = pd.read_csv(f"{path_raw}/contratacoes_ultimos_12_meses.csv", sep=";")

# Base de dados "clientes_desde.csv"
clientes_desde = pd.read_csv(f"{path_raw}/clientes_desde.csv", sep=";")

# Base de dados "historico.csv"
historico = pd.read_csv(f"{path_raw}/historico.csv", sep=";")

# Base de dados "mrr.csv"
mrr = pd.read_csv(f"{path_raw}/mrr.csv", sep=";")

# Base de dados "nps.nps_relacional.csv"
nps_relacional = pd.read_csv(f"{path_raw}/nps_relacional.csv", sep=';')

# Base de dados "nps_transacional_aquisicao.csv"
nps_aquisicao = pd.read_csv(f"{path_raw}/nps_transacional_aquisicao.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_implantacao.csv"
nps_implantacao = pd.read_csv(f"{path_raw}/nps_transacional_implantacao.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_onboarding.csv"
nps_onboarding = pd.read_csv(f"{path_raw}/nps_transacional_onboarding.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_produto.csv"
nps_produto = pd.read_csv(f"{path_raw}/nps_transacional_produto.csv", sep=";", encoding="latin1")

# Base de dados "nps_transacional_suporte.csv"
nps_suporte = pd.read_csv(f"{path_raw}/nps_transacional_suporte.csv", sep=";")

# Ordenação das bases de dados de "telemetria_n.csv"
arquivos_telemetria = sorted(glob.glob(f"{path_raw}/telemetria_*.csv"))

# Base de dados "tickets.csv"
tickets = pd.read_csv(f"{path_raw}/tickets.csv", sep=";")

#### **Transform & Load - Pré-processamento dos dados para a camada "Processed" e PostgreSQL**

##### **Pré-processamento: dados_clientes.csv**

In [ ]:
# Pré-processamento da base de dados "dados_clientes.csv"

# Cópia da base mãe
df_dados_clientes = dados_clientes.copy(deep=True)

# Aplicar nas colunas categóricas principais
colunas_categoricas = [
    "DS_PROD", "DS_LIN_REC", "CIDADE", "DS_CNAE", "DS_SEGMENTO",
    "DS_SUBSEGMENTO", "FAT_FAIXA", "MARCA_TOTVS", "MODAL_COMERC",
    "PERIODICIDADE", "SITUACAO_CONTRATO", "UF"
]

for col in colunas_categoricas:
    df_dados_clientes[col] = padronizar_texto(df_dados_clientes[col])

# Preenchimento de colunas com poucos nulos com 'NAO INFORMADO'
colunas_para_preencher = {
    "DS_SUBSEGMENTO": "NAO INFORMADO",
    "MARCA_TOTVS": "NAO INFORMADO",
    "MODAL_COMERC": "NAO INFORMADO",
    "PERIODICIDADE": "NAO INFORMADO",
    "SITUACAO_CONTRATO": "NAO INFORMADO"
}

for col, valor in colunas_para_preencher.items():
    df_dados_clientes[col] = df_dados_clientes[col].fillna(valor)

# Converter valor do contrato para float
df_dados_clientes["VL_TOTAL_CONTRATO"] = (
    df_dados_clientes["VL_TOTAL_CONTRATO"]
    .replace("[R$ ]", "", regex=True)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
    .round(2)
)

# Faixas de Valor do Contrato
bins = [-float("inf"), 0, 10000, 50000, 100000, 500000, 1000000, float("inf")]
labels = ["ZERADO/NEGATIVO", "ATE 10K", "10K–50K", "50K–100K", "100K–500K", "500K–1M", "ACIMA DE 1M"]
df_dados_clientes["VL_TOTAL_CONTRATO_FAIXA"] = pd.cut(df_dados_clientes["VL_TOTAL_CONTRATO"], bins=bins, labels=labels)

# Converter data de assinatura para datetime
df_dados_clientes["DT_ASSINATURA_CONTRATO"] = pd.to_datetime(df_dados_clientes["DT_ASSINATURA_CONTRATO"], errors="coerce")

# Ano e mês de assinatura
df_dados_clientes["ANO_ASSINATURA"] = df_dados_clientes["DT_ASSINATURA_CONTRATO"].dt.year.astype('int64')
df_dados_clientes["MES_ASSINATURA"] = df_dados_clientes["DT_ASSINATURA_CONTRATO"].dt.month.astype('int64')

# Idade do contrato (em meses)
df_dados_clientes["IDADE_CONTRATO_MESES"] = df_dados_clientes["DT_ASSINATURA_CONTRATO"].apply(idade_contrato_meses)

# Renomear colunas para padrão snake_case
df_dados_clientes.rename(columns={
    "CD_CLIENTE": "cliente_id",
    "DS_PROD": "produto",
    "DS_LIN_REC": "linha_receita",
    "CIDADE": "cidade",
    "DS_CNAE": "cnae",
    "DS_SEGMENTO": "segmento",
    "DS_SUBSEGMENTO": "subsegmento",
    "FAT_FAIXA": "faixa_faturamento",
    "MARCA_TOTVS": "marca_totvs",
    "MODAL_COMERC": "modalidade_comercial",
    "PAIS": "pais",
    "PERIODICIDADE": "periodicidade",
    "SITUACAO_CONTRATO": "situacao_contrato",
    "UF": "uf",
    "VL_TOTAL_CONTRATO": "valor_total_contrato",
    "VL_TOTAL_CONTRATO_FAIXA": "faixa_valor_total_contrato",
    "DT_ASSINATURA_CONTRATO": "data_assinatura_contrato",
    "ANO_ASSINATURA": "ano_assinatura",
    "MES_ASSINATURA": "mes_assinatura",
    "IDADE_CONTRATO_MESES": "idade_contrato_meses"
}, inplace=True)

# Reorganizar colunas
ordem_colunas = [
    "cliente_id", "produto", "linha_receita", "cnae", "segmento", "subsegmento",
    "faixa_faturamento", "marca_totvs", "modalidade_comercial", "periodicidade",
    "situacao_contrato", "cidade", "uf", "pais",
    "valor_total_contrato", "faixa_valor_total_contrato",
    "data_assinatura_contrato", "ano_assinatura", "mes_assinatura", "idade_contrato_meses"
]
df_dados_clientes = df_dados_clientes[ordem_colunas]

# Exportação da base tratada
df_dados_clientes.to_csv(f"{path_processed}/dados_clientes.csv", index=False)

# Carregamento base de dados processada "dados_clientes.csv" no PostreSQL
inserir_dataframe_postgres(df_dados_clientes, "dados_clientes")

##### **Pré-processamento: contratacoes_ultimos_12_meses.csv**

In [ ]:
# Pré-processamento da base de dados "contratacoes_ultimos_12_meses.csv"

# Cópia da base mãe
df_contratacoes = contratacoes.copy(deep=True)

# Conversão do valor para float
df_contratacoes["VLR_CONTRATACOES_12M"] = (
    df_contratacoes["VLR_CONTRATACOES_12M"]
    .replace("[R$ ]", "", regex=True)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
    .round(2)
)

# Renomear colunas para padrão snake_case
df_contratacoes.rename(columns={
    "CD_CLIENTE": "cliente_id",
    "QTD_CONTRATACOES_12M": "qtd_contratacoes_12m",
    "VLR_CONTRATACOES_12M": "vlr_contratacoes_12m"
}, inplace=True)

# Reorganizar colunas
df_contratacoes = df_contratacoes[["cliente_id", "qtd_contratacoes_12m", "vlr_contratacoes_12m"]]

# Exportar base tratada
df_contratacoes.to_csv(f"{path_processed}/contratacoes_ultimos_12_meses.csv", index=False)

# Carregamento base de dados processada "contratacoes_ultimos_12_meses.csv" no PostreSQL
inserir_dataframe_postgres(df_contratacoes, "contratacoes_ultimos_12_meses")

##### **Pré-processamento: clientes_desde.csv**

In [ ]:
# Pré-processamento da base de dados "clientes_desde.csv"

# Cópia da base mãe
df_clientes_desde = clientes_desde.copy(deep=True)

# Renomear colunas
df_clientes_desde.rename(columns={
    "CLIENTE": "cliente_id",
    "CLIENTE_DESDE": "cliente_desde"
}, inplace=True)

# Converter data
df_clientes_desde["cliente_desde"] = pd.to_datetime(df_clientes_desde["cliente_desde"], errors="coerce")

# Criar colunas derivadas
df_clientes_desde["ano_inicio"] = df_clientes_desde["cliente_desde"].dt.year.astype('int64')
df_clientes_desde["tempo_com_empresa_anos"] = (
    (pd.Timestamp("today") - df_clientes_desde["cliente_desde"]).dt.days / 365
).round(1)

# Reorganizar colunas
df_clientes_desde = df_clientes_desde[[
    "cliente_id", "cliente_desde", "ano_inicio", "tempo_com_empresa_anos"
]]

# Exportar base tratada
df_clientes_desde.to_csv(f"{path_processed}/clientes_desde.csv", index=False)

# Carregamento base de dados processada "clientes_desde.csv" no PostreSQL
inserir_dataframe_postgres(df_clientes_desde, "clientes_desde")

##### **Pré-processamento: historico.csv**

In [ ]:
# Pré-processamento da base de dados "historico.csv"

# Cópia da base mãe
df_historico = historico.copy(deep=True)

# Renomeação de colunas
df_historico.rename(columns={
    "NR_PROPOSTA": "nr_proposta",
    "ITEM_PROPOSTA": "item_proposta",
    "DT_UPLOAD": "data_upload",
    "HOSPEDAGEM": "hospedagem",
    "CD_CLI": "cliente_id",
    "FAT_FAIXA": "faixa_faturamento",
    "CD_PROD": "produto_id",
    "QTD": "quantidade",
    "MESES_BONIF": "meses_bonificacao",
    "VL_PCT_DESC_TEMP": "pct_desc_temp",
    "VL_PCT_DESCONTO": "pct_desconto",
    "PRC_UNITARIO": "preco_unitario",
    "VL_DESCONTO_TEMPORARIO": "valor_desc_temp",
    "VL_TOTAL": "valor_total",
    "VL_FULL": "valor_full",
    "VL_DESCONTO": "valor_desconto"
}, inplace=True)

# Padronização de textos
colunas_categoricas = ["faixa_faturamento", "hospedagem"]
for col in colunas_categoricas:
    df_historico[col] = padronizar_texto(df_historico[col])

# Conversão de data
df_historico["data_upload"] = pd.to_datetime(df_historico["data_upload"], errors="coerce")
df_historico["ano_upload"] = df_historico["data_upload"].dt.year.astype('int64')
df_historico["mes_upload"] = df_historico["data_upload"].dt.month.astype('int64')

# Conversão de valores numéricos
colunas_valores = [
    "quantidade", "pct_desc_temp", "pct_desconto", "preco_unitario",
    "valor_desc_temp", "valor_total", "valor_full", "valor_desconto"
]

for col in colunas_valores:
    df_historico[col] = (
        df_historico[col]
        .replace("[R$ ]", "", regex=True)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
        .round(2)
    )

# Reorganizar colunas (exemplo)
colunas_ordenadas = [
    "nr_proposta", "item_proposta", "cliente_id", "produto_id", "data_upload", "ano_upload", "mes_upload",
    "hospedagem", "faixa_faturamento", "quantidade", "meses_bonificacao",
    "pct_desc_temp", "pct_desconto", "preco_unitario", "valor_desc_temp",
    "valor_total", "valor_full", "valor_desconto"
]
df_historico = df_historico[colunas_ordenadas]

# Exportar base tratada
df_historico.to_csv(f"{path_processed}/historico.csv", index=False)

# Carregamento base de dados processada "historico.csv" no PostreSQL
inserir_dataframe_postgres(df_historico, "historico")

##### **Pré-processamento: mrr.csv**

In [ ]:
# Pré-processamento da base de dados "mrr.csv"

# Cópia da base mãe
df_mrr = mrr.copy(deep=True)

# Renomear colunas
df_mrr.rename(columns={
    "CLIENTE": "cliente_id",
    "MRR_12M": "mrr_12m"
}, inplace=True)

# Arredondar MRR para 2 casas decimais
df_mrr["mrr_12m"] = df_mrr["mrr_12m"].round(2)

# Criar faixas de MRR (ajustável com base na distribuição real)
bins = [-0.01, 0, 1000, 5000, 10000, 50000, 100000, float("inf")]
labels = ["ZERADO", "ATE 1K", "1K–5K", "5K–10K", "10K–50K", "50K–100K", "ACIMA DE 100K"]

df_mrr["faixa_mrr_12m"] = pd.cut(df_mrr["mrr_12m"], bins=bins, labels=labels)

# Reorganizar colunas
df_mrr = df_mrr[["cliente_id", "mrr_12m", "faixa_mrr_12m"]]

# Exportar base tratada
df_mrr.to_csv(f"{path_processed}/mrr.csv", index=False)

# Carregamento base de dados processada "mrr.csv" no PostreSQL
inserir_dataframe_postgres(df_mrr, "mrr")

##### **Pré-processamento: nps_relacional.csv**

In [ ]:
# Pré-processamento da base de dados "nps_relacional.csv"

# Cópia da base mãe
df_nps_relacional = nps_relacional.copy(deep=True)

# Renomear colunas para padrão snake_case
df_nps_relacional.rename(columns={
    "respondedAt": "data_resposta",
    "metadata_codcliente": "cliente_id",
    "resposta_NPS": "nota_nps",
    "resposta_unidade": "nota_unidade",
    "Nota_SupTec_Agilidade": "nota_suptec_agilidade",
    "Nota_SupTec_Atendimento": "nota_suptec_atendimento",
    "Nota_Comercial": "nota_comercial",
    "Nota_Custos": "nota_custos",
    "Nota_AdmFin_Atendimento": "nota_admfin_atendimento",
    "Nota_Software": "nota_software",
    "Nota_Software_Atualizacao": "nota_software_atualizacao"
}, inplace=True)

# Conversão de tipos
df_nps_relacional["data_resposta"] = pd.to_datetime(df_nps_relacional["data_resposta"], errors="coerce")
df_nps_relacional["cliente_id"] = df_nps_relacional["cliente_id"].astype(str)
df_nps_relacional["nota_nps"] = df_nps_relacional["nota_nps"].astype('int64')

# Classificação da nota nps
df_nps_relacional["categoria_nps"] = df_nps_relacional["nota_nps"].apply(classificar_nps)

# Demais colunas de nota: garantir float
notas_colunas = [
    "nota_unidade", "nota_suptec_agilidade", "nota_suptec_atendimento",
    "nota_comercial", "nota_custos", "nota_admfin_atendimento",
    "nota_software", "nota_software_atualizacao"
]
df_nps_relacional[notas_colunas] = df_nps_relacional[notas_colunas].astype(float)

# Exportar base tratada
df_nps_relacional.to_csv(f"{path_processed}/nps_relacional.csv", index=False)

# Carregamento base de dados processada "nps_relacional.csv" df_clientes
inserir_dataframe_postgres(df_nps_relacional, "nps_relacional")

##### **Pré-processamento: nps_transacional_aquisicao.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_aquisicao.csv"

# Cópia da base mãe
df_nps_aquisicao = nps_aquisicao.copy(deep=True)

# Renomear colunas para padrão snake_case
df_nps_aquisicao.rename(columns={
    "Cód. Cliente": "cliente_id",
    "Data da Resposta": "data_resposta",
    "Nota NPS": "nota_nps",
    "Nota Agilidade": "nota_agilidade",
    "Nota Conhecimento": "nota_conhecimento",
    "Nota Custo": "nota_custo",
    "Nota Facilidade": "nota_facilidade",
    "Nota Flexibilidade": "nota_flexibilidade"
}, inplace=True)

# Preencher valor nulo em cliente_id
df_nps_aquisicao["cliente_id"] = df_nps_aquisicao["cliente_id"].fillna("NAO INFORMADO").astype(str)

# Conversão de tipos
df_nps_aquisicao["data_resposta"] = pd.to_datetime(df_nps_aquisicao["data_resposta"], errors="coerce")
df_nps_aquisicao["nota_nps"] = df_nps_aquisicao["nota_nps"].astype('int64')

df_nps_aquisicao["categoria_nps"] = df_nps_aquisicao["nota_nps"].apply(classificar_nps)

# Garantir float nas colunas de nota
col_notas = [
    "nota_agilidade", "nota_conhecimento", "nota_custo",
    "nota_facilidade", "nota_flexibilidade"
]
df_nps_aquisicao[col_notas] = df_nps_aquisicao[col_notas].astype(float)

# Exportar base tratada
df_nps_aquisicao.to_csv(f"{path_processed}/nps_transacional_aquisicao.csv", index=False)

# Carregamento base de dados processada "nps_transacional_aquisicao.csv"
inserir_dataframe_postgres(df_nps_aquisicao, "nps_transacional_aquisicao")

##### **Pré-processamento: nps_transacional_implantacao.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_implantacao.csv"

# Cópia da base mãe
df_nps_implatacao = nps_implantacao.copy(deep=True)

# Renomear colunas para padrão snake_case
df_nps_implatacao.rename(columns={
    "Cód. Cliente": "cliente_id",
    "Data da Resposta": "data_resposta",
    "Nota NPS": "nota_nps",
    "Nota Metodologia": "nota_metodologia",
    "Nota Gestao": "nota_gestao",
    "Nota Conhecimento": "nota_conhecimento",
    "Nota Qualidade": "nota_qualidade",
    "Nota Comunicacao": "nota_comunicacao",
    "Nota Prazos": "nota_prazos"
}, inplace=True)

# Preencher cliente_id nulo com 'NAO INFORMADO'
df_nps_implatacao["cliente_id"] = df_nps_implatacao["cliente_id"].fillna("NAO INFORMADO").astype(str)

# Converter data e nota NPS
df_nps_implatacao["data_resposta"] = pd.to_datetime(df_nps_implatacao["data_resposta"], errors="coerce")
df_nps_implatacao["nota_nps"] = df_nps_implatacao["nota_nps"].astype('int64')

# Classificação de NPS
df_nps_implatacao["categoria_nps"] = df_nps_implatacao["nota_nps"].apply(classificar_nps)

# Garantir que demais colunas estejam como float
col_notas = [
    "nota_metodologia", "nota_gestao", "nota_conhecimento",
    "nota_qualidade", "nota_comunicacao", "nota_prazos"
]
df_nps_implatacao[col_notas] = df_nps_implatacao[col_notas].astype(float)

# Exportar base tratada
df_nps_implatacao.to_csv(f"{path_processed}/nps_transacional_implantacao.csv", index=False)

# Carregamento base de dados processada "nps_transacional_implantacao.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_implatacao, "nps_transacional_implantacao")

##### **Pré-processamento: nps_transacional_onboarding.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_onboarding.csv"

# Cópia da base mãe
df_nps_onboarding = nps_onboarding.copy(deep=True)

# Renomear colunas para snake_case e nomes mais limpos
df_nps_onboarding.rename(columns={
    "Data de resposta": "data_resposta",
    "Cod Cliente": "cliente_id",
    "Em uma escala de 0 a 10, quanto você recomenda o Onboarding da TOTVS para um amigo ou colega?.": "nota_nps",
    "Em uma escala de 0 a 10, o quanto você acredita que o atendimento CS Onboarding ajudou no início da sua trajetória com a TOTVS?": "nota_ajuda_inicio",
    "- Duração do tempo na realização da reunião de Onboarding;": "nota_tempo_reuniao",
    "- Clareza no acesso aos canais de comunicação da TOTVS;": "nota_clareza_comunicacao",
    "- Clareza nas informações em geral transmitidas pelo CS que lhe atendeu no Onboarding;": "nota_clareza_informacao",
    "- Expectativas atendidas no Onboarding da TOTVS.": "nota_expectativa"
}, inplace=True)

# Preencher cliente_id nulo com 'NAO INFORMADO'
df_nps_onboarding["cliente_id"] = df_nps_onboarding["cliente_id"].fillna("NAO INFORMADO").astype(str)

# Converter data
df_nps_onboarding["data_resposta"] = pd.to_datetime(df_nps_onboarding["data_resposta"], errors="coerce")

# Garantir que nota_nps seja int
df_nps_onboarding["nota_nps"] = df_nps_onboarding["nota_nps"].astype('int64')

# Criar categoria NPS
df_nps_onboarding["categoria_nps"] = df_nps_onboarding["nota_nps"].apply(classificar_nps)

# Garantir que demais notas sejam float
colunas_float = [
    "nota_ajuda_inicio", "nota_tempo_reuniao", "nota_clareza_comunicacao",
    "nota_clareza_informacao", "nota_expectativa"
]
df_nps_onboarding[colunas_float] = df_nps_onboarding[colunas_float].astype(float)

# Exportar base tratada
df_nps_onboarding.to_csv(f"{path_processed}/nps_transacional_onboarding.csv", index=False)

# Carregamento base de dados processada "nps_transacional_onboarding.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_onboarding, "nps_transacional_onboarding")

##### **Pré-processamento: nps_transacional_produto.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_produto.csv"

# Cópia da base mãe
df_nps_produto = nps_produto.copy(deep=True)

# Renomear colunas
df_nps_produto.rename(columns={
    "Data da Resposta": "data_resposta",
    "Linha de Produto": "linha_produto",
    "Nome do Produto": "nome_produto",
    "Nota": "nota_nps",
    "Cód. T": "cliente_id"
}, inplace=True)

# Converter data
df_nps_produto["data_resposta"] = pd.to_datetime(df_nps_produto["data_resposta"], errors="coerce")

# Padronizar colunas de texto (linha e nome do produto)
df_nps_produto["linha_produto"] = padronizar_texto(df_nps_produto["linha_produto"])
df_nps_produto["nome_produto"] = padronizar_texto(df_nps_produto["nome_produto"])

# Garantir que cliente_id seja string
df_nps_produto["cliente_id"] = df_nps_produto["cliente_id"].astype(str)

# Garantir nota como int
df_nps_produto["nota_nps"] = df_nps_produto["nota_nps"].astype('int64')

# Classificação de NPS
df_nps_produto["categoria_nps"] = df_nps_produto["nota_nps"].apply(classificar_nps)

# Exportar base tratada
df_nps_produto.to_csv(f"{path_processed}/nps_transacional_produto.csv", index=False)

# Carregamento base de dados processada "nps_transacional_produto.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_produto, "nps_transacional_produto")

##### **Pré-processamento: nps_transacional_suporte.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_suporte.csv"

# Cópia da base mãe
df_nps_suporte = nps_suporte.copy(deep=True)

# Renomear colunas
df_nps_suporte.rename(columns={
    "ticket": "ticket_id",
    "resposta_NPS": "nota_nps",
    "grupo_NPS": "categoria_nps",
    "Nota_ConhecimentoAgente": "nota_conhecimento_agente",
    "Nota_Solucao": "nota_solucao",
    "Nota_TempoRetorno": "nota_tempo_retorno",
    "Nota_Facilidade": "nota_facilidade",
    "Nota_Satisfacao": "nota_satisfacao",
    "cliente": "cliente_id"
}, inplace=True)

# Padronizar a categoria NPS
df_nps_suporte["categoria_nps"] = df_nps_suporte["categoria_nps"].str.upper().str.strip()
df_nps_suporte["categoria_nps"] = df_nps_suporte["categoria_nps"].replace("PASSIVO", "NEUTRO")


# Garantir tipos consistentes
df_nps_suporte["ticket_id"] = df_nps_suporte["ticket_id"].astype('int64')
df_nps_suporte["nota_nps"] = df_nps_suporte["nota_nps"].astype('int64')
df_nps_suporte["cliente_id"] = df_nps_suporte["cliente_id"].astype(str)

# Exportar base tratada
df_nps_suporte.to_csv(f"{path_processed}/nps_transacional_suporte.csv", index=False)

# Carregamento base de dados processada "nps_transacional_suporte.csv" no PostgreSQL
inserir_dataframe_postgres(df_nps_suporte, "nps_transacional_suporte")

##### **Pré-processamento: telemetria_1.csv a telemetria_11.csv**

In [ ]:
# Pré-processamento das bases de dados "telemetria_n.csv"

for arquivo_telemetria in arquivos_telemetria:
    df_telemetria = pd.read_csv(arquivo_telemetria)

    # Conversão de datas
    df_telemetria['referencedatestart'] = pd.to_datetime(df_telemetria['referencedatestart'], errors='coerce')

    # Criar coluna de referência no formato YYYY e MM
    df_telemetria["ano_referencia"] = df_telemetria["referencedatestart"].dt.year.astype('int64')
    df_telemetria["mes_referencia"] = df_telemetria["referencedatestart"].dt.month.astype('int64')

    # Padronização de texto
    df_telemetria['statuslicenca'] = df_telemetria['statuslicenca'].astype(str).str.strip().str.upper()

    # Preencher valores nulos em 'statuslicenca' com 'NAO INFORMADO'
    df_telemetria['statuslicenca'] = df_telemetria['statuslicenca'].replace('NAN', np.nan)
    df_telemetria['statuslicenca'] = df_telemetria['statuslicenca'].fillna('NAO INFORMADO')

    # Remover colunas irrelevantes (tcloud e clienteprime, pois são totalmente nulas)
    df_telemetria.drop(columns=['tcloud', 'clienteprime'], inplace=True)

    # Renomeação de colunas
    df_telemetria.rename(columns={
        'clienteid': 'cliente_id',
        'eventduration': 'duracao_evento',
        'moduloid': 'modulo_id',
        'productlineid': 'linha_produto_id',
        'referencedatestart': 'data_referencia',
        'slotid': 'slot_id',
        'statuslicenca': 'status_licenca'
    }, inplace=True)

    # Reorganização das colunas
    ordem_colunas = [
        'cliente_id', 'data_referencia', 'ano_referencia', 'mes_referencia', 'duracao_evento',
        'modulo_id', 'linha_produto_id', 'slot_id', 'status_licenca'
    ]
    df_telemetria = df_telemetria[ordem_colunas]

    # Exportar base tratada
    df_telemetria.to_csv(f"{path_processed}/{arquivo_telemetria.split("\\")[1]}", index=False)
    
    # Carregamento base de dados processada "telemetria_[n].csv" no PostgreSQL
    inserir_dataframe_postgres(df_telemetria, f"telemetria_{re.findall(r'\d+', arquivo_telemetria)[0]}")

##### **Pré-processamento: tickets.csv**

In [ ]:
# Pré-processamento da base de dados "nps_transacional_suporte.csv"

# Cópia da base mãe
df_tickets = tickets.copy(deep=True)

# Conversão de datas
df_tickets['DT_CRIACAO'] = pd.to_datetime(df_tickets['DT_CRIACAO'], errors='coerce')
df_tickets['DT_ATUALIZACAO'] = pd.to_datetime(df_tickets['DT_ATUALIZACAO'], errors='coerce')

# Criação de colunas de granularidade temporal
df_tickets['ANO_CRIACAO'] = df_tickets['DT_CRIACAO'].dt.year.astype('int64')
df_tickets['MES_CRIACAO'] = df_tickets['DT_CRIACAO'].dt.month.astype('int64')

# Padronização de texto (strip + upper)
colunas_texto = ['NOME_GRUPO', 'TIPO_TICKET', 'STATUS_TICKET', 'PRIORIDADE_TICKET']
for col in colunas_texto:
    df_tickets[col] = df_tickets[col].str.strip().str.upper().apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", errors="ignore").decode("utf-8"))

# Renomear colunas para snake_case
df_tickets.rename(columns={
    'CODIGO_ORGANIZACAO': 'codigo_organizacao',
    'NOME_GRUPO': 'nome_grupo',
    'TIPO_TICKET': 'tipo_ticket',
    'STATUS_TICKET': 'status_ticket',
    'DT_CRIACAO': 'data_criacao',
    'DT_ATUALIZACAO': 'data_atualizacao',
    'BK_TICKET': 'ticket_id',
    'PRIORIDADE_TICKET': 'prioridade_ticket',
    'ANO_CRIACAO': 'ano_criacao',
    'MES_CRIACAO': 'mes_criacao'
}, inplace=True)

# Reorganização das colunas
ordem_colunas = [
    'ticket_id', 'codigo_organizacao', 'nome_grupo', 'tipo_ticket', 'status_ticket',
    'prioridade_ticket', 'data_criacao', 'ano_criacao', 'mes_criacao', 'data_atualizacao'
]
df_tickets = df_tickets[ordem_colunas]

# Exportação da base tratada
df_tickets.to_csv(f"{path_processed}/tickets.csv", index=False)

# Carregamento base de dados processada "tickets.csv" no PostgreSQL
inserir_dataframe_postgres(df_tickets, "tickets")